# 1. Building Abstractions with Procedures
## 1.1 The Elements of Programming
First section of the chapter is primarily focused on syntax of Scheme, I will not take any notes regarding that topics, however there is one interesting example in the book.

### Square Roots by Newton's Method
Using recursion as iteration because there is NO loop structure in Scheme!!! First we define a `sqrt-iter` method, it checks if the guess is close enough to the target, if so, it will return the guess otherwise call itself again with an improved version of the guess.
```scheme
(define (sqrt-iter guess target)
  (if (good-enough? guess target) guess (sqrt-iter (improve guess target) target)))
```

`good-enough?` is an approximation check
```scheme
(define (good-enough? guess target)
  (< (abs (- (square guess) target)) 0.0001))
```

A guess is improved by averaging it with the target
```scheme
(define (improve guess target)
  (average guess (/ target guess)))

(define (average x y)
  (/ (+ x y) 2))
```

Finally, the `sqrt` method is defined as follows, using 1 as the initial guess
```scheme
(define (sqrt num)
  (sqrt-iter 1.0 num))
```

In [6]:
def sqrt_iter(guess, target):
    if is_good_enough(guess, target):
        return guess
    return sqrt_iter(improve(guess, target), target)

def is_good_enough(guess, target):
    return abs(guess**2 - target) < 0.0001

def improve(guess, target):
    return average(guess, target/guess)

def average(x, y):
    return (x + y) / 2.0

def sqrt(num):
    return sqrt_iter(1.0, num)

sqrt(2)

1.4142156862745097

### Cube Roots by Netwon's Method
```scheme
(define (cube-root-iter guess target)
  (if (cube-good-enough? guess target) guess (cube-root-iter (improve-cube-guess guess target) target)))

(define (improve-cube-guess guess target)
  (/ (+ (/ target (square guess)) (* 2 guess)) 3))

(define (cube-good-enough? guess target)
  (< (abs (- (* (square guess) guess) target)) 0.0001))

(define (cube-root num)
  (cube-root-iter 1.0 num))
```

We can improve our solution using internal definition and block structure
```scheme
(define (sqrt x)
  (define (good-enough? guess)
    (< (abs (- (square guess) x)) 0.001))
  (define (improve guess)
    (average guess (/ x guess)))
  (define (sqrt-iter guess)
    (if (good-enough? guess) guess (sqrt-iter (improve guess))))
  (sqrt-iter 1.0))
```

## 1.2 Procedures and Processes They Generate
### Recursions
Think about factorials, the classic example of recursion. There are actually two ways to go about doing it recursively. 

This is a linear recursive process 
```scheme
(define (factorial n)
  (if (= n 1) 1 (* n (factorial(- n 1)))))
```

Now this is a linear iterative process
```scheme
(define (factorial n)
  (fact-iter 1 1 n))

(define (fact-iter product counter max-count)
  (if (> counter max-count) product
    (fact-iter (* counter product) (+ counter 1) max-count)))
```

Notice the difference? The second method keeps its internal state. Here is another example:

```scheme
(define (exp base n)
  (exp-iter base n 1))

(define (exp-iter base counter product)
  (if (= counter 0) product (exp-iter base (- counter 1) (* base product))))

(exp 10 4)
```

### Key Idea
In contrasting iteration and recursion, we must be careful not to confuse the notion of a recursive process with the notion of a recursive procedure. When we describe the procedure as recurisve, we are referring to the syntactic fact that the procedure definition refers to the procedure itself. But when we describe a process as following a pattern that is, say, linearly recursive, we are speaking about how the process evolves, not about the syntax of how a procedure is written. It may seem disturbing that we refer a recursive procedure such as `fact-iter` as generating an iterative process. However the process really is iterative: Its state is captured completely by its three state variables, and an interpreter need keep track of only three variables in order to execute the process.


### Counting Change
The number of ways to change the amount `a` using `n` kinds of coins equals to the sum of the following:
1. the number of ways to change amount `a` using all BUT the first kind of coin
2. the number of ways to change amount `a - d` using all `n` kinds of coins, where `d` is the denomination of the first kind of coin.


To see why this is true, the ways to make change can be dvided into two groups; those that do not use any of the first kind of coin and those that do.

* If `a` is exactly 0, we should count that as 1 way to make change
* If `a` is less than 0, we should count that as 0 ways to make change
* If `n` is 0, we should count that as 0 ways to make change

```scheme
(define (count-change amount)
  (cc amount 5))

(define (cc amount kinds-of-coins)
  (cond ((= amount 0) 1)
        ((or (< amount 0) (= kinds-of-coins 0)) 0)
        (else (+ (cc amount (- kinds-of-coins 1)) (cc (- amount (first-denomination kinds-of-coins)) kinds-of-coins)))))

(define (first-denomination kinds-of-coins)
  (cond ((= kinds-of-coins 1) 1)
        ((= kinds-of-coins 2) 5)
        ((= kinds-of-coins 3) 10)
        ((= kinds-of-coins 4) 25)
        ((= kinds-of-coins 5) 50)))
```

### Testing for Primality
#### Greatest Common Divisor
If `r` is the remainder of `a/b`, then the common divisors of `a` and `b` are precisely the same as the common divisors of `b` and `r`. Thus, this problem is reducible until remainder is 0. 

```
gcd(a, b) = gcd(b, r) 
gcd(206, 40) = gcd(40, 6) = gcd(6, 4) = gcd(4, 2) = gcd(2, 0), thus gcd is 2
```

```scheme
(define (gcd a b)
  (if (= b 0) a (gcd b (remainder a b))))
```

In [2]:
def greatest_common_divisor(a, b):
    if b == 0:
        return a
    return greatest_common_divisor(b, a % b)

greatest_common_divisor(40, 10)

10

#### Sqrt(n) Solution
```scheme
(define (smallest-divisor n)
  (find-divisor n 2))

(define (find-divisor n test-divisor)
  (cond ((> (square test-divisor) n) n)
        ((divides? test-divisor n) test-divisor)
        (else (find-divisor n (+ test-divisor 1)))))

(define (divides? a b)
  (= (remainder a b) 0))

(define (prime? n)
  (= n (smallest-divisor n)))
```

#### The Fermat Test Solution
Later

## 1.3 Formulating Abstractions with Higher-Order Procedures
This is equivalent to using higher order function to create new function which is commonly used in Redux. Suppose we are tasked with creating a summation function such that we can perform things like

$$
\frac{1}{1 \cdot 3}\frac{1}{5 \cdot 7}\frac{1}{9 \cdot 11}... = \frac{\pi}{8}
$$

Then we should create a higher order function that performs summation for us:
```scheme
(define (sum term a next b)
  (if (> a b)
    0
    (+ (term a) (sum term (next a) next b))))
```

Create a `term` function that takes a parameter `a` and performs `1 / ((a + 2) * a)`.
```scheme
(define (pi-term a)
  (/ 1.0 (* a (+ a 2))))
```

Create a `pi-next` function that makes an increment of 4 on a given input. 
```scheme
(define (pi-next n)
  (+ n 4))
```

Put it together
```scheme
(define (pi-sum a b)
  (sum pi-term a next b))
```

In [8]:
def summation(term, a, next_increment, b):
    if a > b:
        return 0
    return term(a) + summation(term, next_increment(a), next_increment, b)

def pi_term(n):
    return 1.0 / ((n + 2) * n)

def pi_next(n):
    return n + 4

def pi_sum(a, b):
    return summation(pi_term, a, pi_next, b)
    
print pi_sum(1, 1000)

0.392449081949


### Expressing Integral
$$
\int f = (f(a + \frac{dx}{2}) + f(a + \frac{dx}{2} + dx) + f(a + \frac{dx}{2} + 2dx))dx
$$

We can reuse our summation function above and construct the following function:
```scheme
(define (integral f a b dx)
  (define (add-dx x) (+ x dx))
  (* (sum f (+ a (/ dx 2.0)) add-dx b) dx))
```

### Lambda
Lambda is a convenient way to define procedure annoymously. 
```scheme
(lambda (x) (+ x 4))
```

The `pi-sum` function could have been implemented using lambdas.
```scheme
(define (pi-sum a b)
  (sum (lambda (x) (/ 1.0 (* x (+ x 2)))) a (lambda (x) (+ x 4)) b))
```

Similarly, the same can be done with integral.
```scheme
(define (integral f a b dx)
  (* (sum f (+ a (/ dx 2.0)) (lambda (x) (+ x dx)) b) dx))
```


### Using `let` to create local variables
Suppose we want to compute the following function

$$
f(x,y) = x(1 + xy)^{2} + y(1-y) + (1 + xy)(1 - y)
$$

We can express each term with a local variable

$$
a = 1 + xy \\
b = 1 - y \\
f(x, y) = xa^{2} + yb + ab
$$

We can use `let`
```scheme
(define (f x y)
  (let ((a (+ 1 (* x y)))
        (b (- 1 y)))
    (+ (* x (square a))
       (* y b)
       (* a b)))
```

The general form of a `let` expression is
```
(let ((<var_1> <exp_1>)
      (<var_2> <exp_2>)
      ...
      (<var_n> <exp_n>))
      body)
```

* `let` allows one to bind variables as locally as possible to where they are to be used. For example, if the value of x is 5, the value of the following expression is 38.
```scheme
(+ (let ((x 3)) (+ x (* x 10)))
    x)
```

* The variables' values are computed outside the `let`. For example, if the value of x is 2, the following expression will have the value 12.
```scheme
(let ((x 3)
      (y (+ x 2))
      (* x y))
```

### Finding fixed points of functions
A number x is called fixed point of a function `f` if x satisfies the equation

$$
f(x) = x
$$

For some functions `f`, we can locate a fixed point by beginning with an initial guess and applying f repeatedly until the value does not change very much.

$$
f(x), \;f(f(x)), \;f(f(f(x))), ...
$$

Using this idea, we can come up with a procedure `fixed-point` that produces an approximation to a fixed point of a function.
```scheme
(define tolerance 0.00001)
(define (fixed-point f first-guess)
  (define (close-enough? v1 v2)
      (< (abs (- v1 v2)) tolerance))
  (define (try guess)
    (let ((next (f guess)))
      (if (close-enough? guess next) 
          next
          (try next))))
  (try first-guess))
```

Using what we have, we can find a solution to the equation 

$$
y = sin(y) + cos(y)
$$

```scheme
(fixed-point (lambda (y) (+ (sin y) (cos y)))
             1.0)
```

### Procedures as Returned Values
We have seen that we can pass procedures as arguments and obviously we can definitely use procedures as returned values. This is a common feature in many modern language, I am surprised to find that even Scheme has it. 

Let's express the idea of average damping in a procedure that takes a procedure `f` as argument and returns a lambda that uses the procedure `f`.
```scheme
(define (average-damp f)
  (lambda (x) (average x (f x))))
```

For example, applying `average-damp` to `square` function produces a procedure whose value at some number x is the average of `x` and `x^2`. 
```scheme
((average-damp square) 10)
```

Using `average-damp`, we can reformulate the square-root procedure as follows:
```scheme
(define (sqrt x)
  (fixed-point (average-damp (lambda (y) (/ x y)))
               1.0))
```